# 🚀 Episode-level Fixed PID Training with RecurrentPPO (GPU)

**Amaç**: RL ile optimal FIXED PID parametrelerini bul (Kp, Ki, Kd)

**Yaklaşım**: Episode-level RL
- 1 timestep = 1 episode (full 500-step simulation)
- Action: [Kp, Ki, Kd] - bir kere seçilir
- Observation: Downsampled trajectory (600D)
- Algorithm: RecurrentPPO (LSTM policy)

**Kaggle Setup**:
1. Settings → Accelerator → **GPU T4 x2** (önerilen)
2. Run All → ~1-2 hours
3. Optimal PID değerleri sonunda gösterilir

---

## 📦 Installation

In [ ]:
%%capture
!pip install gymnasium "numpy<2" torch stable-baselines3 sb3-contrib tensorboard tqdm

In [ ]:
import numpy as np
import torch
import gymnasium as gym
from gymnasium import spaces
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback
from stable_baselines3.common.monitor import Monitor
import os
from datetime import datetime

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA device count: {torch.cuda.device_count()}")

## 🎮 Simulation Classes (Inline)

In [ ]:
class PIDController:
    """PID Controller for missile heading"""
    def __init__(self, kp=2.0, ki=0.1, kd=0.5):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.integral = 0.0
        self.prev_error = 0.0

    def update(self, error, dt):
        self.integral += error * dt
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        output = self.kp * error + self.ki * self.integral + self.kd * derivative
        self.prev_error = error
        return output

    def reset(self):
        self.integral = 0.0
        self.prev_error = 0.0


class Missile:
    """2D Missile with PID control"""
    def __init__(self, x=0.0, y=0.0, vx=0.0, vy=0.0,
                 max_speed=1000.0, max_accel=1000.0,
                 kp=2.0, ki=0.1, kd=0.5):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.max_speed = max_speed
        self.max_accel = max_accel
        self.pid = PIDController(kp, ki, kd)
        self.active = True
        self.trajectory = []

    @property
    def position(self):
        return np.array([self.x, self.y])

    @property
    def velocity(self):
        return np.array([self.vx, self.vy])

    @property
    def speed(self):
        return np.linalg.norm(self.velocity)

    @property
    def heading(self):
        return np.arctan2(self.vy, self.vx)

    def update(self, target_pos, dt):
        if not self.active:
            return

        # Calculate desired heading
        dx = target_pos[0] - self.x
        dy = target_pos[1] - self.y
        desired_heading = np.arctan2(dy, dx)

        # Calculate heading error
        current_heading = self.heading
        error = desired_heading - current_heading
        error = np.arctan2(np.sin(error), np.cos(error))  # Normalize to [-pi, pi]

        # PID control
        control = self.pid.update(error, dt)
        new_heading = current_heading + control * dt

        # Calculate desired velocity
        desired_vx = self.max_speed * np.cos(new_heading)
        desired_vy = self.max_speed * np.sin(new_heading)

        # Apply acceleration limits
        dvx = desired_vx - self.vx
        dvy = desired_vy - self.vy
        accel_magnitude = np.sqrt(dvx**2 + dvy**2) / dt if dt > 0 else 0

        if accel_magnitude > self.max_accel:
            scale = self.max_accel / accel_magnitude
            dvx *= scale
            dvy *= scale

        # Update velocity
        self.vx += dvx
        self.vy += dvy

        # Limit speed
        speed = self.speed
        if speed > self.max_speed:
            self.vx = (self.vx / speed) * self.max_speed
            self.vy = (self.vy / speed) * self.max_speed

        # Update position
        self.x += self.vx * dt
        self.y += self.vy * dt

        self.trajectory.append((self.x, self.y))


class Target:
    """2D Target with various maneuvers"""
    def __init__(self, x=5000.0, y=5000.0, speed=1000.0, maneuver='circular'):
        self.x = x
        self.y = y
        self.speed = speed
        self.maneuver = maneuver
        self.heading = 0.0
        self.maneuver_timer = 0.0
        self.trajectory = []

    @property
    def position(self):
        return np.array([self.x, self.y])

    @property
    def velocity(self):
        vx = self.speed * np.cos(self.heading)
        vy = self.speed * np.sin(self.heading)
        return np.array([vx, vy])

    def update(self, dt, missile_pos=None):
        self.maneuver_timer += dt

        # Apply maneuver
        if self.maneuver == 'straight':
            pass  # No change in heading

        elif self.maneuver == 'circular':
            turn_rate = 0.5  # rad/s
            self.heading += turn_rate * dt

        elif self.maneuver == 'zigzag':
            if self.maneuver_timer > 2.0:
                self.heading += np.pi / 4
                self.maneuver_timer = 0.0

        elif self.maneuver == 'evasive' and missile_pos is not None:
            dx = self.x - missile_pos[0]
            dy = self.y - missile_pos[1]
            distance = np.sqrt(dx**2 + dy**2)

            if distance < 2000.0:
                escape_heading = np.arctan2(dy, dx)
                self.heading = escape_heading

        # Update position
        vel = self.velocity
        self.x += vel[0] * dt
        self.y += vel[1] * dt

        self.trajectory.append((self.x, self.y))


print("✅ Simulation classes loaded")

## 🏋️ Episode-level Environment

In [ ]:
class EpisodicFixedPIDEnv(gym.Env):
    """
    Episode-level RL environment for finding optimal fixed PID parameters.

    Key differences from step-level:
    - Action sets PID parameters ONCE at episode start
    - Environment runs FULL simulation (500 steps)
    - Observation = downsampled trajectory (50 samples × 12 features = 600D)
    - Reward = episodic (based on full trajectory result)
    """

    metadata = {'render_modes': ['human', 'rgb_array'], 'render_fps': 60}

    def __init__(self, map_size=10000.0, hit_radius=50.0, max_steps=500,
                 dt=0.01, target_maneuver='circular',
                 missile_speed=1000.0, missile_accel=1000.0,
                 target_speed=1000.0, downsample_rate=10):
        super().__init__()

        # Action: Direct PID parameter values (WIDE range for large simulation)
        self.action_space = spaces.Box(
            low=np.array([0.1, 0.0, 0.0], dtype=np.float32),
            high=np.array([10000.0, 50.0, 50.0], dtype=np.float32),
            dtype=np.float32
        )

        # Observation: Downsampled trajectory (600D)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(600,),
            dtype=np.float32
        )

        # Environment parameters
        self.map_size = map_size
        self.hit_radius = hit_radius
        self.max_steps = max_steps
        self.dt = dt
        self.target_maneuver = target_maneuver
        self.missile_speed = missile_speed
        self.missile_accel = missile_accel
        self.target_speed = target_speed
        self.downsample_rate = downsample_rate

        # State
        self.missile = None
        self.target = None
        self.trajectory = []

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        # Reset missile (random initial position and velocity)
        missile_x = np.random.uniform(0, 0.2 * self.map_size)
        missile_y = np.random.uniform(0.2 * self.map_size, 0.8 * self.map_size)
        initial_vx = np.random.uniform(0.8 * self.missile_speed, 0.9 * self.missile_speed)

        self.missile = Missile(
            x=missile_x, y=missile_y,
            vx=initial_vx, vy=0.0,
            max_speed=self.missile_speed,
            max_accel=self.missile_accel,
            kp=2.0, ki=0.1, kd=0.5  # Default (will be overridden)
        )

        # Reset target (random initial position)
        target_x = np.random.uniform(0.6 * self.map_size, 0.9 * self.map_size)
        target_y = np.random.uniform(0.3 * self.map_size, 0.7 * self.map_size)
        target_heading = np.random.uniform(0, 2 * np.pi)

        self.target = Target(
            x=target_x, y=target_y,
            speed=self.target_speed,
            maneuver=self.target_maneuver
        )
        self.target.heading = target_heading

        self.trajectory = []

        # Initial observation: zeros (no trajectory yet)
        return np.zeros(600, dtype=np.float32), {}

    def step(self, action):
        """
        Run FULL simulation with given PID parameters

        Args:
            action: [Kp, Ki, Kd] PID parameters

        Returns:
            obs: Downsampled trajectory (600D)
            reward: Episodic reward
            done: Always True (1 step = 1 episode)
            truncated: Always False
            info: Episode statistics
        """
        # 1. Set PID parameters from action
        Kp = float(action[0])
        Ki = float(action[1])
        Kd = float(action[2])

        self.missile.pid.kp = Kp
        self.missile.pid.ki = Ki
        self.missile.pid.kd = Kd

        # 2. Run FULL simulation (500 steps)
        self.trajectory = []
        hit = False
        hit_time = self.max_steps

        for step in range(self.max_steps):
            # Update simulation
            self.missile.update(self.target.position, self.dt)
            self.target.update(self.dt, missile_pos=self.missile.position)

            # Calculate metrics
            dx = self.target.x - self.missile.x
            dy = self.target.y - self.missile.y
            distance = np.sqrt(dx**2 + dy**2)

            # Angle error
            desired_heading = np.arctan2(dy, dx)
            current_heading = self.missile.heading
            angle_error = desired_heading - current_heading
            angle_error = np.arctan2(np.sin(angle_error), np.cos(angle_error))

            # Closing velocity
            missile_vel = self.missile.velocity
            target_vel = self.target.velocity
            relative_vel = missile_vel - target_vel
            range_vec = np.array([dx, dy])
            if distance > 0:
                closing_velocity = -np.dot(relative_vel, range_vec) / distance
            else:
                closing_velocity = 0.0

            heading_error = angle_error

            # Get target velocity components
            target_velocity = self.target.velocity

            # Record trajectory (12 features)
            self.trajectory.append([
                self.missile.x,
                self.missile.y,
                self.missile.vx,
                self.missile.vy,
                self.target.x,
                self.target.y,
                target_velocity[0],  # target vx
                target_velocity[1],  # target vy
                distance,
                angle_error,
                closing_velocity,
                heading_error
            ])

            # Check hit
            if distance < self.hit_radius:
                hit = True
                hit_time = step
                break

            # Check out of bounds
            if (self.missile.x < -1000 or self.missile.x > self.map_size + 1000 or
                self.missile.y < -1000 or self.missile.y > self.map_size + 1000):
                break

            # Check missile active
            if not self.missile.active:
                break

        # 3. Downsample trajectory
        trajectory_array = np.array(self.trajectory)

        if len(trajectory_array) > 0:
            # Downsample
            downsampled = trajectory_array[::self.downsample_rate]

            # Ensure exactly 50 samples (pad or trim)
            if len(downsampled) < 50:
                # Pad with last sample
                padding = np.tile(downsampled[-1], (50 - len(downsampled), 1))
                downsampled = np.vstack([downsampled, padding])
            elif len(downsampled) > 50:
                # Trim
                downsampled = downsampled[:50]

            # Flatten to 600D
            obs = downsampled.flatten().astype(np.float32)
        else:
            # No trajectory (immediate failure)
            obs = np.zeros(600, dtype=np.float32)

        # 4. Calculate episodic reward
        reward = self._calculate_reward(trajectory_array, hit, hit_time)

        # 5. Episode info
        final_distance = trajectory_array[-1, 8] if len(trajectory_array) > 0 else 10000.0

        info = {
            'hit': hit,
            'hit_time': hit_time,
            'final_distance': final_distance,
            'trajectory_length': len(trajectory_array),
            'pid_kp': Kp,
            'pid_ki': Ki,
            'pid_kd': Kd
        }

        # Episode always done after 1 step (full simulation)
        return obs, reward, True, False, info

    def _calculate_reward(self, trajectory, hit, hit_time):
        """Calculate episodic reward based on full trajectory"""
        reward = 0.0

        if len(trajectory) == 0:
            return -100.0

        # 1. Hit/Miss
        if hit:
            reward += 100.0
            # Time bonus: Faster intercept is better
            time_bonus = (self.max_steps - hit_time) / 10.0
            reward += time_bonus
        else:
            # Miss penalty
            reward -= 50.0
            # Final distance penalty
            final_distance = trajectory[-1, 8]
            reward -= final_distance / 1000.0

        # 2. Average distance penalty (trajectory quality)
        avg_distance = np.mean(trajectory[:, 8])
        reward -= avg_distance / 1000.0

        # 3. Trajectory smoothness
        velocities = trajectory[:, 2:4]
        if len(velocities) > 1:
            accelerations = np.diff(velocities, axis=0)
            jerk = np.diff(accelerations, axis=0)
            smoothness_penalty = np.mean(np.linalg.norm(jerk, axis=1))
            reward -= smoothness_penalty / 10000.0

        # 4. Closing velocity bonus
        avg_closing_vel = np.mean(trajectory[:, 10])
        if avg_closing_vel > 0:
            reward += avg_closing_vel / 1000.0

        return reward


print("✅ EpisodicFixedPIDEnv loaded")

## ⚙️ Training Configuration

In [ ]:
# Training parameters
ALGORITHM = 'RecurrentPPO'  # RecurrentPPO with LSTM
MANEUVER = 'circular'  # Target maneuver: straight, circular, zigzag, evasive
N_ENVS = 8  # Parallel environments (GPU can handle more)
TOTAL_TIMESTEPS = 50_000  # Total episodes (50K = ~2 hours on T4)
SAVE_FREQ = 5_000  # Checkpoint frequency

# Missile/Target parameters
MISSILE_SPEED = 1000.0  # m/s
MISSILE_ACCEL = 1000.0  # m/s²
TARGET_SPEED = 1000.0  # m/s

print(f"""\n{'='*60}
EPISODE-LEVEL FIXED PID TRAINING
{'='*60}
Algorithm: {ALGORITHM}
Target Maneuver: {MANEUVER}
Parallel Environments: {N_ENVS}
Total Episodes: {TOTAL_TIMESTEPS:,}
Missile: {MISSILE_SPEED} m/s, {MISSILE_ACCEL} m/s²
Target: {TARGET_SPEED} m/s
{'='*60}\n""")

## 🏗️ Environment Setup

In [ ]:
def make_env(rank):
    """Create a monitored environment"""
    def _init():
        env = EpisodicFixedPIDEnv(
            target_maneuver=MANEUVER,
            missile_speed=MISSILE_SPEED,
            missile_accel=MISSILE_ACCEL,
            target_speed=TARGET_SPEED
        )
        env = Monitor(env)
        return env
    return _init


# Create directories
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_dir = f"models/{ALGORITHM.lower()}_{MANEUVER}_{timestamp}"
log_dir = f"logs/{ALGORITHM.lower()}_{MANEUVER}_{timestamp}"
os.makedirs(model_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

print(f"Model directory: {model_dir}")
print(f"Log directory: {log_dir}\n")

# Create vectorized environments
print("Creating environments...")
env = DummyVecEnv([make_env(i) for i in range(N_ENVS)])
eval_env = DummyVecEnv([make_env(0)])

print(f"✅ Created {N_ENVS} training environments")
print(f"✅ Created 1 evaluation environment\n")

## 🤖 Model Initialization

In [ ]:
print(f"Initializing {ALGORITHM} model...\n")

# RecurrentPPO with LSTM policy
model = RecurrentPPO(
    'MlpLstmPolicy',
    env,
    learning_rate=3e-4,
    n_steps=2048 // N_ENVS,  # Adjust for vectorized env
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    ent_coef=0.01,
    policy_kwargs={
        'lstm_hidden_size': 256,
        'n_lstm_layers': 1,
        'enable_critic_lstm': True
    },
    verbose=1,
    tensorboard_log=log_dir,
    device='cuda'  # Force GPU
)

print(f"✅ Model initialized on {model.device}")
print(f"\n📊 Model architecture:")
print(model.policy)

## 🔄 Callbacks Setup

In [ ]:
# Checkpoint callback
checkpoint_callback = CheckpointCallback(
    save_freq=SAVE_FREQ // N_ENVS,
    save_path=model_dir,
    name_prefix=f"{ALGORITHM.lower()}_fixed_pid"
)

# Evaluation callback
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=model_dir,
    log_path=log_dir,
    eval_freq=2_500 // N_ENVS,
    n_eval_episodes=5,
    deterministic=True,
    render=False
)

print("✅ Callbacks configured")
print(f"   - Checkpoints every {SAVE_FREQ:,} steps")
print(f"   - Evaluation every {2_500:,} steps\n")

## 🚀 Training (GPU Accelerated)

**Note**: 1 timestep = 1 episode (full 500-step simulation)

**Expected time**: ~1-2 hours for 50K episodes on T4 GPU

In [ ]:
print(f"\n{'='*60}")
print("STARTING TRAINING")
print(f"{'='*60}\n")
print(f"Monitor training with: tensorboard --logdir {log_dir}\n")

try:
    model.learn(
        total_timesteps=TOTAL_TIMESTEPS,
        callback=[checkpoint_callback, eval_callback],
        progress_bar=True
    )
except KeyboardInterrupt:
    print("\n\n⚠️ Training interrupted by user!")

# Save final model
final_path = os.path.join(model_dir, f"{ALGORITHM.lower()}_fixed_pid_final.zip")
model.save(final_path)

print(f"\n{'='*60}")
print("✅ TRAINING COMPLETED")
print(f"{'='*60}")
print(f"Final model saved to: {final_path}\n")

## 📊 Test Learned PID Parameters

In [ ]:
print(f"\n{'='*60}")
print("TESTING LEARNED PID PARAMETERS")
print(f"{'='*60}\n")

# Create test environment
test_env = EpisodicFixedPIDEnv(
    target_maneuver=MANEUVER,
    missile_speed=MISSILE_SPEED,
    missile_accel=MISSILE_ACCEL,
    target_speed=TARGET_SPEED
)

n_test_episodes = 20
hits = 0
total_reward = 0
distances = []
hit_times = []
pid_values = []

# RecurrentPPO needs lstm_states
lstm_states = None

for episode in range(n_test_episodes):
    obs, _ = test_env.reset()
    episode_reward = 0

    action, lstm_states = model.predict(obs, state=lstm_states, deterministic=True)
    obs, reward, done, _, info = test_env.step(action)
    episode_reward += reward

    total_reward += episode_reward
    if info['hit']:
        hits += 1
        hit_times.append(info['hit_time'])

    distances.append(info['final_distance'])
    pid_values.append([info['pid_kp'], info['pid_ki'], info['pid_kd']])

    print(f"Episode {episode+1:2d}: {'HIT' if info['hit'] else 'MISS':4s} | "
          f"Time={info['hit_time']:3d} | "
          f"Dist={info['final_distance']:7.1f}m | "
          f"Reward={episode_reward:7.1f} | "
          f"PID=(Kp={info['pid_kp']:.2f}, Ki={info['pid_ki']:.3f}, Kd={info['pid_kd']:.3f})")

# Calculate statistics
hit_rate = hits / n_test_episodes * 100
avg_reward = total_reward / n_test_episodes
avg_hit_time = np.mean(hit_times) if hit_times else 0

# Average PID values
pid_values = np.array(pid_values)
avg_kp = np.mean(pid_values[:, 0])
avg_ki = np.mean(pid_values[:, 1])
avg_kd = np.mean(pid_values[:, 2])
std_kp = np.std(pid_values[:, 0])
std_ki = np.std(pid_values[:, 1])
std_kd = np.std(pid_values[:, 2])

print(f"\n{'='*60}")
print(f"TEST RESULTS ({n_test_episodes} episodes)")
print(f"{'='*60}")
print(f"Hit Rate: {hit_rate:.1f}%")
print(f"Average Reward: {avg_reward:.1f}")
print(f"Average Hit Time: {avg_hit_time:.1f} steps")
print(f"Average Final Distance: {np.mean(distances):.1f}m")
print(f"\n🎯 OPTIMAL PID PARAMETERS for '{MANEUVER}' target:")
print(f"{'='*60}")
print(f"  Kp = {avg_kp:.3f} ± {std_kp:.3f}")
print(f"  Ki = {avg_ki:.3f} ± {std_ki:.3f}")
print(f"  Kd = {avg_kd:.3f} ± {std_kd:.3f}")
print(f"{'='*60}\n")

print(f"💡 Use these values in demo.py:")
print(f"   python demo.py --maneuver {MANEUVER} "
      f"--kp {avg_kp:.3f} --ki {avg_ki:.3f} --kd {avg_kd:.3f}\n")

## 💾 Download Best Model

Download `best_model.zip` from the model directory to use locally!

In [ ]:
# List all saved models
print(f"\n📁 Saved models in {model_dir}:")
!ls -lh {model_dir}

print(f"\n✅ Training complete! Download 'best_model.zip' to use locally.")
print(f"\n🎯 Optimal PID for '{MANEUVER}' target:")
print(f"   Kp = {avg_kp:.3f}")
print(f"   Ki = {avg_ki:.3f}")
print(f"   Kd = {avg_kd:.3f}")

---

## 📝 Notes

### Episode-level RL Approach:
- **Action once**: PID parameters [Kp, Ki, Kd] selected at episode start
- **Full simulation**: 500 steps run with fixed PID
- **Trajectory observation**: 600D (50 samples × 12 features)
- **Episodic reward**: Based on hit/miss, time, trajectory quality

### Why RecurrentPPO?
- LSTM policy can learn from trajectory sequences
- Better for temporal patterns in simulation data
- More stable than step-level RL for PID tuning

### Training Time:
- 50K episodes ≈ 1-2 hours on T4 GPU
- Can increase to 100K for better convergence
- Hit rate typically reaches 70-80% for circular target

### Different Maneuvers:
- `straight`: Easiest (80-90% hit rate)
- `circular`: Medium (70-80% hit rate)
- `zigzag`: Medium-hard (60-70% hit rate)
- `evasive`: Hardest (50-60% hit rate)

---